# Year to year ERA prediction

This notebook analyzes which statistics best predict a pitchers Earned Run Average, (ERA) from one year to the next

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import sklearn.metrics

%matplotlib inline

We first import select pitching data for qualified starting pitchers from www.fangraphs.com in the date range 2002-2017, for an explanation of each statistic see https://www.fangraphs.com/library/pitching/ .

In [3]:
stats=pd.read_csv('FanGraphs Leaderboard.csv')

In [3]:
current =pd.read_csv('pitch2019.csv')

In [10]:
stats['Name']=' '+stats['Name']

In [3]:
predseasons=['exitv'+str(i) for i in range(2015,2020)]
i=2015
for season in predseasons:
  
    globals()[season] = pd.read_csv('exit_velocitypitch'+str(i)+'.csv')
    globals()[season]['Name']=globals()[season][' first_name']+' '+globals()[season]['last_name']
    i+=1

In [4]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2002,2018)]
i=2002
for season in predseasons:
  
    globals()[season] = pd.merge(stats[stats['Season']==i],stats[stats['Season']==(i+1)][['Name','ERA']],on="Name")
    i+=1
   

In [7]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2015,2018)]
i=2015
for season in predseasons:
  
    globals()[season] = pd.merge(globals()[season],globals()['exitv'+str(i)][['avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance','ev95percent', 'brl_percent', 'brl_pa', 'Name']],on="Name")
    i+=1



In [14]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2002,2015)]
statsrel=pd.concat([globals()[season] for season in predseasons])


In [44]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2015,2018)]
statspost=pd.concat([globals()[season] for season in predseasons])

In [18]:

statsrel=statsrel.rename(columns={'ERA_x':'ERA_current_year','ERA_y':'ERA_next_year'})
statsrel=statsrel.reset_index(drop=True)
for col in statsrel.columns:
    if type(statsrel[col][1])==str:
        statsrel[col]=statsrel[col].str.replace('%','',regex=False)


In [95]:
current=current.rename(columns={'ERA':'ERA_current_year'})
current=current.reset_index(drop=True)
for col in current.columns:
    if type(current[col][1])==str:
        current[col]=current[col].str.replace('%','',regex=False)

In [13]:
statsrel.columns

NameError: name 'statsrel' is not defined

One sees that there is a column for each statistic and a row for each pitcher for each season

The included stats are listed above, I have only included rate stats because counting statistics will skew predicitions in favor of pitchers who have pitched more innings in a given season.

Below I reformat the data values in each column so that they are all floats and are suitable for analysis

In [96]:
for col in current.columns:
    if type(current[col][1])==str:
        current[col]=current[col].str.replace('%','',regex=False)

In [12]:
statsrel.tail()

,Season,Name,Team,Age,ERA_current_year,K/9,BB/9,K/BB,H/9,HR/9,...,max_hit_speed,avg_hit_speed,fbld,gb,max_distance,avg_distance,avg_hr_distance,ev95percent,brl_percent,brl_pa
91,2017,German Marquez,Rockies,22,4.39,8.17,2.72,3.00,9.67,1.39,...,113.2,88.0,92.5,84.6,462,173,410.0,36.0,7.1,5.0
92,2017,Julio Teheran,Braves,26,4.49,7.22,3.44,2.10,8.89,1.48,...,111.5,86.2,91.7,81.6,433,182,397.0,30.2,6.4,4.6
93,2017,Tanner Roark,Nationals,30,4.57,8.32,3.23,2.57,8.78,1.13,...,113.9,86.2,90.8,82.6,433,163,401.0,32.8,5.4,3.7
94,2017,Rick Porcello,Red Sox,28,4.65,8.01,2.12,3.77,10.45,1.68,...,114.3,87.9,92.6,84.0,465,186,398.0,35.2,8.0,5.9
95,2017,Kevin Gausman,Orioles,26,4.68,8.63,3.42,2.52,10.03,1.40,...,119.4,87.3,92.0,83.5,450,175,405.0,33.6,8.2,5.6


In [19]:
numbers=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'WHIP', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'K%', 'BB%',
       'Soft%', 'Med%', 'Hard%', 'AVG', 'IFH%', 'BUH%', 'xFIP', 'SIERA',
       'RS/9', 'K-BB%', 'Pull%', 'Cent%', 'Oppo%', 'playerid', 'ERA_next_year']
for col in numbers:
    statsrel[col]=statsrel[col].astype(float)
    
    
    
    


In [97]:
numbers=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'WHIP', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'K%', 'BB%',
       'Soft%', 'Med%', 'Hard%', 'AVG', 'IFH%', 'BUH%', 'xFIP', 'SIERA',
       'RS/9', 'K-BB%', 'Pull%', 'Cent%', 'Oppo%', 'avg_hit_angle', 'anglesweetspotpercent', 'max_hit_speed',
       'avg_hit_speed', 'fbld', 'gb', 'max_distance', 'avg_distance',
       'avg_hr_distance', 'ev95percent', 'brl_percent', 'brl_pa','playerid']
for col in numbers:
    current[col]=current[col].astype(float)
    

KeyError: 'avg_hit_angle'

In the next three cells I create a new dataframe from the original that includes a column for the given pitchers ERA the following season, and rename the columns accordingly

"features" is a list of statistics that we will use to try to predict a pitchers ERA.

"featureswithestimators" also includes the ERA estimators xFIP (eXpected Fielding Independent Pitching) and SIERA (Skill Interactive ERA), these stats use strikeout, walk and fly ball rates to say what a pitchers ERA 'should' be by removing factors such as team defense that are out of the pitchers control. For more information on these stats see https://www.fangraphs.com/library/pitching/. 

In [20]:
features=[ 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'WHIP', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'K%', 'BB%',
       'Soft%', 'Med%', 'Hard%', 'AVG', 'IFH%', 'BUH%','RS/9','Pull%', 'Cent%', 'Oppo%']


In [99]:
featureswithstatcast=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'WHIP', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'K%', 'BB%',
       'Soft%', 'Med%', 'Hard%', 'AVG', 'IFH%', 'BUH%','RS/9','Pull%', 'Cent%', 'Oppo%','avg_hit_angle', 'anglesweetspotpercent', 'max_hit_speed',
       'avg_hit_speed', 'fbld', 'gb', 'max_distance', 'avg_distance',
       'avg_hr_distance', 'ev95percent', 'brl_percent', 'brl_pa']

In [54]:
featureswithestimators=['Age', 'ERA_current_year', 'K/9', 'BB/9',
       'K/BB', 'H/9', 'HR/9', 'WHIP', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%',
       'IFFB%', 'HR/FB', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'K%', 'BB%',
       'Soft%', 'Med%', 'Hard%', 'AVG', 'IFH%', 'BUH%', 'xFIP', 'SIERA',
       'RS/9', 'K-BB%', 'Pull%', 'Cent%', 'Oppo%','avg_hit_angle', 'anglesweetspotpercent', 'max_hit_speed',
       'avg_hit_speed', 'fbld', 'gb', 'max_distance', 'avg_distance',
       'avg_hr_distance', 'ev95percent', 'brl_percent', 'brl_pa']

First lets explore which stats are most correlated with a pitcher's ERA during the same year

In [21]:
corrdict1={col:np.abs(statsrel[col].corr(statsrel['ERA_current_year'])) for col in featureswithestimators}



    
Corr=pd.DataFrame.from_dict(corrdict1, orient='index')
Corr.sort_values(by=0,ascending=False)

NameError: name 'featureswithestimators' is not defined

We see that WHIP and hits per nine innings have strong correlations to ERA, placing just above xFIP and SIERA as the most strongly correlated variables. Now lets take a look at how these stats are correlated to a pitcher's ERA the following season.

In [23]:
corrdict2={col:np.abs(statsrel[col].corr(statsrel['ERA_next_year'])) for col in features}



    
Corr=pd.DataFrame.from_dict(corrdict2, orient='index')
Corr.sort_values(by=0,ascending=False)

,0
K%,0.443530
K/9,0.424201
ERA_current_year,0.359383
SwStr%,0.334764
AVG,0.332433
K/BB,0.331993
WHIP,0.329655
Contact%,0.329361
H/9,0.328516
Z-Contact%,0.305747


   Here we see a very different picture, SIERA and xFIP have moved to the front of the pack, justifying their use as truer measures of a pitchers talent than ERA alone. We see that WHIP, and H/9 are now only the 8th and 10th most correlated stats respectively. 
    
   Besides xFIP and SIERA, it seems that the best raw stats to use to predict a pitchers ERA are the ones involving strikeouts, (K%, K/9,K/BB, SwStr%). Its no surprise then that strikeouts are a major component of both the SIERA and xFIP estimators.
    
   Also of note is that a pitchers walk rate, (BB%) has a very low correlation to his ERA the next year. It seems that walks do not hurt a pitcher very much as long as he maintains high strikeout totals.
     

Finally, we try to use a combination of the raw features in a linear regressor to predict a pitchers ERA the following year.

First we build our design matrix and scale our features accordingly.

In [25]:
X = StandardScaler().fit_transform(statsrel[features])
X.shape

(701, 35)

In [26]:
y= statsrel['ERA_next_year'].values
len(y)

701

 Next we shuffle the order of the data so that the initial ordering does not bias the model.

In [113]:
from sklearn.utils import shuffle
X,y=shuffle(X,y)




Now we can use a linear regressor on the data to try and make predictions. We test our model using 5 fold cross validation, and evaluate it using the root mean squared error.

In [28]:
regressor = LinearRegression()
MSE=cross_val_score(regressor, X,y, cv=5,scoring='neg_mean_squared_error').mean()
(-MSE)**.5


0.7247861354796373

We see that our model is off by an average of about .726 runs in predicting a pitcher's ERA. For comparison, lets look at our errors if we use the pitchers ERA from the previous year as our predicition.

In [117]:

sklearn.metrics.r2_score(y_true = statsrel['ERA_next_year'], y_pred = statsrel['ERA_current_year'])

-0.3249315891270921

Our regressor fares significantly better, how about if we use SIERA and xFIP?


In [29]:
RMSESIERA = np.sqrt(mean_squared_error(y_true = statsrel['ERA_next_year'], y_pred = statsrel['SIERA']))
RMSESIERA
#sklearn.metrics.explained_variance_score(y_true = statsrel['ERA_next_year'], y_pred = statsrel['SIERA'])

0.7534396635936927

In [30]:
RMSExFIP = np.sqrt(mean_squared_error(y_true = statsrel['ERA_next_year'], y_pred = statsrel['xFIP']))
RMSExFIP
sklearn.metrics.r2_score(y_true = statsrel['ERA_next_year'], y_pred = statsrel['xFIP'])

0.13666580911727289

Our regressor fares slightly better than xFIP and SIERA, which is no surprise given they both largely use stats from our list of features as inputs.




We can improve our Regression model further by incorporating a regularization scheme, this will help keep the model from overfitiing the data. Lets first try a Ridge regressor, which helps keep the coefficients of our features small. 

In [120]:
from sklearn.linear_model import Ridge,Lasso

regressor=Ridge(alpha=300)
MSE=cross_val_score(regressor, X,y, cv=5,scoring='neg_mean_squared_error').mean()
(-MSE)**.5



0.7012168111979183

In [30]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

We see a slight improvement in our MSE from earlier (.714 vs .717). Next lets try a Lasso regressor, this will make the algorithm concentrate on a smaller number of features.

In [122]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
regressor=Lasso(alpha=.02)
MSE=cross_val_score(regressor, X,y, cv=5,scoring='neg_mean_squared_error').mean()
(-MSE)**.5



0.7001537832236034

We find a similar improvement to our model.



One may wonder how I chose the specific parameters for my Regularization models, this is done using grid search. Grid search performs the regression task for a given parameter range and outputs the parameter that leads to the best performing model. An example of grid search for a Ridge Regression model is given below:

In [34]:
from sklearn.model_selection import GridSearchCV
regressor=Ridge()
parameters = {'alpha': [1,10,100,1000]}
gs_reg = GridSearchCV(regressor,parameters, cv=5, iid=False, n_jobs=-1) 
gs_reg = gs_reg.fit(X, y)
for param_name in sorted(parameters.keys()):
    print('Optimal parameter'+' '+"%s: %r" % (param_name, gs_reg.best_params_[param_name]))


Optimal parameter alpha: 100


Finally, lets look at the coefficients of our model to see which stats most strongly influence its prediction. (We use our Ridge regression model)

In [51]:
regressor=Ridge(alpha=250)
np.sort(np.abs(regressor.fit(X,y).coef_))


array([0.00054071, 0.00142527, 0.0025592 , 0.0049953 , 0.00703772,
       0.00803927, 0.01159724, 0.01305153, 0.01312508, 0.0135203 ,
       0.01382633, 0.01427469, 0.0145463 , 0.01705308, 0.01711234,
       0.01725207, 0.01887015, 0.02084751, 0.02223594, 0.02268413,
       0.0246783 , 0.02471869, 0.03168543, 0.03362303, 0.03727409,
       0.03792837, 0.04430651, 0.04566185, 0.05096121, 0.05706926,
       0.05828168, 0.07429094, 0.07511735, 0.07906548])

In [123]:
np.sort(np.abs(regressor.fit(X,y).coef_))

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00397749, 0.00514772, 0.00797082, 0.01350535,
       0.01665086, 0.01810559, 0.01913338, 0.03728872, 0.05644182,
       0.06317586, 0.07535095, 0.09024062, 0.10147453, 0.18030747])

Our top three coefficents are .109, .097, and .085, lets see which stats these correspond to

In [124]:
most=np.argsort(np.abs(regressor.fit(X,y).coef_))[:]
for i in reversed([statsrel[features].columns[i] for i in most]):
    print(i)

K%
Z-Contact%
HR/9
K/BB
GB/FB
Cent%
Pull%
ERA_current_year
O-Contact%
Med%
F-Strike%
IFFB%
IFH%
Soft%
Zone%
LD%
K/9
BB/9
RS/9
H/9
BUH%
WHIP
BABIP
SwStr%
GB%
Contact%
FB%
BB%
HR/FB
O-Swing%
Z-Swing%
Swing%
AVG
Hard%
Oppo%


Given our correlation analysis above and the success of xFIP and SIERA it is unsurprising that K/9 and K/BB are among the strongest predictors of future performance. Interestingly, Z-Contact% (the rate at which a batter makes contact on pitches in the strike zone against a given pitcher) has a higher coefficient than we we would expect. 

In [60]:
XC=StandardScaler().fit_transform(current[features])

In [61]:
regressor=Ridge(alpha=300)
regressor.fit(X,y)
current['preds']=regressor.predict(XC)

In [62]:
preds=current[['Name','ERA_current_year','SIERA','preds']].sort_values('preds')


In [63]:
preds.to_csv('pitching_leaders.csv')

In [3]:
leaders=pd.read_csv('pitching_leaders.csv')

In [11]:
leaders[20:]

,Unnamed: 0,Name,ERA_current_year,SIERA,preds,diff
20,102,Noah Syndergaard,4.90,3.80,3.451254,1.448746
21,59,Jacob deGrom,3.71,3.53,3.456905,0.253095
22,6,Jake Odorizzi,2.16,4.22,3.483013,-1.323013
23,55,Robbie Ray,3.59,4.20,3.498995,0.091005
24,26,Luke Weaver,3.03,3.65,3.500460,-0.470460
25,73,Walker Buehler,4.03,3.91,3.512157,0.517843
26,110,Eduardo Rodriguez,5.04,4.01,3.525834,1.514166
27,78,Tyler Mahle,4.15,3.70,3.541810,0.608190
28,9,Justin Verlander,2.38,3.24,3.556361,-1.176361
29,44,Max Fried,3.28,3.77,3.566391,-0.286391


In [4]:
leaders['diff']=leaders['ERA_current_year']-leaders['preds']

In [7]:
leaders.sort_values('diff', ascending=False)[leaders['preds']<4.0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Unnamed: 0,Name,ERA_current_year,SIERA,preds,diff
52,146,Freddy Peralta,7.36,4.41,3.707202,3.652798
93,139,Derek Holland,6.75,4.54,3.994996,2.755004
81,134,Drew Pomeranz,6.45,4.66,3.908243,2.541757
57,125,Corey Kluber,5.80,4.51,3.757869,2.042131
58,118,Kevin Gausman,5.56,4.47,3.761343,1.798657
59,116,Yu Darvish,5.40,4.82,3.761729,1.638271
26,110,Eduardo Rodriguez,5.04,4.01,3.525834,1.514166
20,102,Noah Syndergaard,4.90,3.80,3.451254,1.448746
60,114,Carlos Rodon,5.19,4.01,3.763701,1.426299
76,113,Cal Quantrill,5.14,4.52,3.867018,1.272982


In [125]:
import pickle
regressor=Lasso(alpha=.02)
regressor.fit(X,y)
pickle.dump(regressor,open('era_predict.pkl','wb'))